In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install stanza

In [ ]:
!pip install transformers
!pip install mlflow --quiet
!pip install pyngrok --quiet

In [ ]:
import os 
import torch 
#import stanza
import pickle 
import pandas as pd 
import seaborn as sns
from tqdm import tqdm
from time import time
import seaborn as sn
from matplotlib import pyplot as plt 
import numpy as np 
import datetime
import random 
from shutil import copytree
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, LongformerTokenizer, \
    LongformerForSequenceClassification, BertForSequenceClassification, AdamW,\
    get_cosine_schedule_with_warmup, BertTokenizerFast, BertModel
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, RandomSampler, DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
from torch.nn import CrossEntropyLoss, LSTM
import torch
from sklearn.metrics import max_error, balanced_accuracy_score
import torch 
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as Opt
#torch.set_default_tensor_type(torch.HalfTensor)
#torch.set_default_tensor_type(torch.cuda.HalfTensor)
import mlflow
from pyngrok import ngrok

## Lemmatization:

In [ ]:
stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize, lemma')

In [ ]:
# 2347/2858 
for file_name in tqdm(os.listdir(config['paths']['scripts'])):
    imdb_id = file_name.split('_')[1].replace('.txt', '')
    if not os.path.exists(os.path.join(config['paths']['script_lemmas'], file_name.replace('.txt','_lemmas.txt'))):
        text = read_script(os.path.join(config['paths']['scripts'], file_name))
        try:
            doc = nlp(text)
        except Exception as e:
            print(e)
            continue
        doc_sentences = []
        for sentence in doc.sentences:
            doc_sentences.append(' '.join([word.lemma for word in sentence.words]))    
        processed_text = ' '.join(doc_sentences)
        with open(os.path.join(config['paths']['script_lemmas'], file_name.replace('.txt','_lemmas.txt')), 'w', encoding='utf-8') as f:
                f.write(processed_text)
    

In [ ]:
print(len(os.listdir(config['paths']['script_lemmas'])))

In [ ]:
def process_with_stanza(imdb_id_to_texts, lang='en', processors='tokenize, lemma'):
    '''
    Args:
        reviews (list of str): raw reviews
        lang (str): language
        processors (str): preprocessing functions
    '''
    stanza.download('en')
    nlp = stanza.Pipeline(lang=lang, processors=processors)
    #processed_texts = read_processed_texts()
    processed_texts = {}
    
    for imdb_id, text in tqdm(imdb_id_to_texts.items()):
        try:
            doc = nlp(text)
        except Exception as e:
            print(e)
            continue
        doc_sentences = []

        for sentence in doc.sentences:
            doc_sentences.append(' '.join([word.lemma for word in sentence.words]))
            
        processed_texts[imdb_id] = ' '.join(doc_sentences)
        #processed_texts.append(' '.join(doc_sentences))
    with open(os.path.join(config['paths']['script_lemmas'], , 'wb') as f:
        pickle.dump(processed_texts, f)

    return processed_reviews

# Regression (bert-based):

In [ ]:
## initialize model object and print this object to see the model

In [ ]:
config = { 
        'paths':
         { 
            'scripts' : '/content/drive/MyDrive/NLP/Movie scripts dataset/Movie scripts and annotations/Scripts',
            'movies_matching_scores' : '/content/drive/MyDrive/NLP/Movie scripts dataset/Movie characters/Matching evaluation and statistics/movies_mean_matching_scores.xlsx',
            'logs_dir':'/content/drive/MyDrive/NLP/Movie scripts models/BERTScorer/logs',
            'ckpt_dir': '/content/drive/MyDrive/NLP/Movie scripts models/BERTScorer/ckpts',
            'script_annotations': '/content/drive/MyDrive/NLP/Movie scripts dataset/Movie scripts and annotations/Script annotations by BERT/row_classification',
            #'manual_annotations': '/content/drive/MyDrive/NLP/Movie scripts dataset/Movie scripts and annotations/Script manual annotations',
             'mlruns': '/content/drive/MyDrive/NLP/Movie scripts models/BERTScorer/mlruns',
             'movie_meta_data': '/content/drive/MyDrive/NLP/Movie scripts dataset/Movie meta data/movie_meta_data.csv',
             'script_lemmas':'/content/drive/MyDrive/NLP/Movie scripts dataset/Movie scripts and annotations/Scripts_lemmas',
             'meta_score_exampels': '/content/drive/MyDrive/NLP/Movie scripts dataset/Baseline models/MetaScoreAssessorExamples'
        },
        'train': {
                'optim' : {
                    'AdamW':{
                        'lr':1e-4#,
                        #'eps': 1e-8,
                        #'weight_decay':0.0001
                    }
                },
                #'grad_accum_iters': 2, 
                'embedding_size': 768, 
                'model': BertForSequenceClassification, # from [BertForSequenceClassification, LongformerForSequenceClassification]
                'tokenizer':BertTokenizerFast, # from [BertTokenizer, LongformerTokenizer]
                'pretrained_model_type': 'bert-base-cased', # from ['bert-base-cased', 'allenai/longformer-base-4096']
                 'scene_as_chunk': True, 
                #'max_script_length': 65540 , #  131072, 65540, 32770
                'max_seq_length': 512,
                'max_scene_number' : 100, 
                'num_classes' : 1,
                'nrof_steps' : 1000, 
                'nrof_epochs' : 200, 
                'tr_batch_size' : 4,
                'tst_batch_size' : 4,
                'exp_name':'script_scorer', 
                'device': torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
                'load_model':True, 
                'save_model':True 
                }
        }

In [ ]:
tokenizer = config['train']['tokenizer'].from_pretrained(config['train']['pretrained_model_type'], 
                                          do_lower_case=False)

## Dataset preprocessing:

### structures:

In [ ]:
def read_script(path):
    try:
        with open(path, 'r') as f:
            text = f.read()
            
    except:
        with open(path, 'r', encoding='latin-1') as f:
            text = f.read()

    return text 

def show_histogram(x_data, y_data, values_to_show=None, figsize=(40, 10), x_label='x', y_label='y',
                   set_rotation=False, title='title', file_path='name.png', dpi=100, to_save=False, to_show=True):

    fig = plt.figure(figsize=figsize)
    h = sns.barplot(x=x_data, y=y_data, palette="Blues_d")
    if values_to_show:
        for i,y in enumerate(y_data):
            h.text(i, y, str(y)+'\n('+str(values_to_show[i]) + ')', color='black', ha='center')
    if set_rotation:
        h.set_xticklabels(h.get_xticklabels(), rotation=45)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    if to_show:
        plt.show()

    if to_save:
        fig.savefig(file_path, dpi=dpi, bbox_inches='tight')

def remove_labels(row):
    if row.startswith('text:'):
        row = row[6:]
    elif row.startswith('dialog:'):
        row = row[8:]
    elif row.startswith('speaker_heading:'):
        row = row[17:]
    elif row.startswith('scene_heading:'):
        row = row[15:]
    return row.strip()


In [ ]:
class ScriptData:
    def __init__(self, config, tokenizer):
        self.config = config 
        self.tokenizer = tokenizer
        self.movie_meta_dict = self.read_movie_meta_data()
        self.scripts, self.labels, self.imdb_ids = self._get_scripts_and_labels(to_load=True)
        self.scripts_scenes = self.get_script_scenes()
    
    def read_movie_meta_data(self):
        movie_meta_df = pd.read_csv(self.config['paths']['movie_meta_data'])
        movie_meta_dict = dict(zip(movie_meta_df['imdbid'].tolist(), 
                                   movie_meta_df['metascore'].tolist()))
        
        return movie_meta_dict
    

    def _get_scripts_and_labels(self, to_load=False, from_anno=False):
        if not (to_load and os.path.exists('scripts.pickle') and os.path.exists('labels.pickle')):
            scripts, labels, imdb_ids = [], [], []
            script_iterator_files = self.config['paths']['script_annotations'] if self.config['train']['scene_as_chunk'] else self.config['paths']['scripts']
            #for file_name in tqdm(os.listdir(self.config['paths']['scripts'])):
            for file_name in tqdm(os.listdir(script_iterator_files)):
                imdb_id = file_name.split('_')[1].replace('.txt', '')
                if int(imdb_id) in self.movie_meta_dict:
                    if self.movie_meta_dict[int(imdb_id)]!=-1:
                        with open(os.path.join(script_iterator_files, file_name), 'r') as f:
                            anno_lines = f.readlines()
                        #text = read_script(os.path.join(self.config['paths']['scripts'], file_name))
                        scripts.append(anno_lines)
                        labels.append(float(self.movie_meta_dict[int(imdb_id)]) / 100.)
                        imdb_ids.append(imdb_id)
                        #labels.append(int(self.movie_meta_dict[int(imdb_id)]>=60))
            with open('scripts.pickle', 'wb') as f:
                pickle.dump(scripts, f)
            with open('labels.pickle', 'wb') as f:
                pickle.dump(labels, f)
            with open('imdb_ids.pickle', 'wb') as f:
                pickle.dump(imdb_ids, f)
            print('Data saved')
        else:
            with open('scripts.pickle', 'rb') as f:
                scripts = pickle.load(f)
            with open('labels.pickle', 'rb') as f:
                labels = pickle.load(f)
            with open('imdb_ids.pickle', 'rb') as f:
                imdb_ids = pickle.load(f)
            print('Data loaded')
        
        return scripts, labels, imdb_ids
    
    '''
    def tokenize_scripts(self, to_load=True):
        if to_load and os.path.exists('tokenized_scripts.pickle'):
            with open('tokenized_scripts.pickle', 'rb') as f:
                tokenized_scripts = pickle.load(f)
        else:
            tokenized_scripts = self.tokenizer(self.scripts)['input_ids']
            tokenized_scripts = [x[1:] for x in tokenized_scripts] # remove cls 
            with open('tokenized_scripts.pickle', 'wb') as f:
                pickle.dump(tokenized_scripts, f)
        return tokenized_scripts
    '''
    def prepare_tokenized_chunks_masks_labels(self, 
                                              tokenized_script_chunks, 
                                              attention_masks,
                                              labels):
        tokenized_script_chunks = [torch.tensor(chunk) for chunk in tokenized_script_chunks]
        attention_masks = [torch.tensor(mask) for mask in attention_masks]
        padded_tokenized_script_chunks = pad_sequence(tokenized_script_chunks, 
                                                      padding_value=0, batch_first=True)# for BERT pad = 0 ?
        padded_attention_masks = pad_sequence(attention_masks, 
                                              padding_value=0, batch_first=True)
        labels = torch.tensor(labels, dtype=torch.float)
        #labels = torch.LongTensor(labels)

        return padded_tokenized_script_chunks, padded_attention_masks, labels

    def get_script_scenes(self):
        scripts_scenes = []

        scene_text = ''
        for i, script_lines in enumerate(self.scripts):
            scripts_scenes.append([])
            for line_num, line in enumerate(script_lines):
                if line.startswith('scene_heading:'):
                    if scene_text:
                        scripts_scenes[-1].append(scene_text)
                    scene_text = remove_labels(line)
                else:
                    scene_text+=remove_labels(line)
            if not scripts_scenes[-1]:
                scripts_scenes.pop(-1)
                self.labels.pop(i)
                self.imdb_ids.pop(i)

        return scripts_scenes

    def make_tokenized_chunks_for_scripts(self, to_load=True):
        if not (to_load and os.path.exists('tokenized_script_chunks.pickle') and os.path.exists('attention_masks.pickle')):
            tokenized_script_chunks, attention_masks = [], []
            text_iterator = self.scripts_scenes if self.config['train']['scene_as_chunk'] else self.scripts 

            for script_scenes in tqdm(text_iterator):
                if len(script_scenes)>0:
                    tokenized_scenes = self.tokenizer(script_scenes, truncation=True, 
                                                    padding='max_length',
                                                    return_attention_mask=True) #['input_ids']
                    tokenized_scenes_ids = tokenized_scenes['input_ids'][:self.config['train']['max_scene_number']]
                    tokenized_scenes_attention_masks = tokenized_scenes['attention_mask'][:self.config['train']['max_scene_number']]
                    tokenized_script_chunks.append(tokenized_scenes_ids)
                    attention_masks.append(tokenized_scenes_attention_masks)
            with open('tokenized_script_chunks.pickle', 'wb') as f:
                pickle.dump(tokenized_script_chunks, f)
            with open('attention_masks.pickle', 'wb') as f:
                pickle.dump(attention_masks, f)
        else:
            with open('tokenized_script_chunks.pickle', 'rb') as f:
                tokenized_script_chunks = pickle.load(f)
            with open('attention_masks.pickle', 'rb') as f:
                attention_masks = pickle.load(f)
                
        return tokenized_script_chunks, attention_masks



    '''
    def make_tokenized_chunks_for_scripts(self, to_load=False):
        tokenized_scripts = self.tokenize_scripts()
        tokenized_script_chunks, attention_masks = [], []
        max_script_l = self.config['train']['max_script_length']
        max_seq_l = self.config['train']['max_seq_length']
        n = int(max_script_l / (max_seq_l//2)) - 1
        
        for t_script in tqdm(tokenized_scripts):    
            t_script = t_script[:max_script_l]
            n = int(len(t_script) / (max_seq_l))
            script_chunks_masks = [(t_script[i*(max_seq_l):i*(max_seq_l) + max_seq_l - 1], [1]*(min(max_seq_l, len(t_script) - i*(max_seq_l - 1)))) for i in range(n)]

            if len(script_chunks_masks[-1][0]) < max_seq_l - 1 :
                script_chunks_masks[-1] = (script_chunks_masks[-1][0] + [1] * (max_seq_l - 1 - len(script_chunks_masks[-1][0])),
                                           script_chunks_masks[-1][1] + [0] * (max_seq_l - 1 - len(script_chunks_masks[-1][1])))
            
            script_chunks, script_masks = zip(*script_chunks_masks)
            script_chunks = [[101] + x for x in script_chunks]
            text_masks = [[1] + x for x in text_masks]
            tokenized_script_chunks.append(script_chunks)
            attention_masks.append(script_masks)

        return tokenized_scripts, tokenized_script_chunks, attention_masks
    '''
    def get_train_val_split(self):
        #_, tokenized_script_chunks, attention_masks = self.make_tokenized_chunks_for_scripts()
        tokenized_script_chunks, attention_masks = self.make_tokenized_chunks_for_scripts()
        
        tr_inputs, val_inputs, tr_masks, val_masks, tr_labels, val_labels, tr_imdb_ids, val_imdb_ids = train_test_split(
            tokenized_script_chunks, attention_masks,  
            self.labels, self.imdb_ids, 
             test_size=0.2, random_state=11)
        tst_inputs, val_inputs, tst_masks, val_masks, tst_labels, val_labels, tst_imdb_ids, val_imdb_ids = train_test_split(
            val_inputs, val_masks, val_labels, val_imdb_ids, 
            test_size=0.25, random_state=11)
        classes_info = 'Train set: '+str(len(tr_inputs))+'\nVal set: ' + str(len(val_inputs)) +'\nTest set: '+str(len(tst_inputs))
        with open('dataset_info.txt', 'w') as f:
            f.write(classes_info)
        mlflow.log_artifact('dataset_info.txt')

        tr_inputs, tr_masks, tr_labels = self.prepare_tokenized_chunks_masks_labels(
            tr_inputs, tr_masks, tr_labels)
        val_inputs, val_masks, val_labels = self.prepare_tokenized_chunks_masks_labels(
            val_inputs, val_masks, val_labels)
        tst_inputs, tst_masks, tst_labels = self.prepare_tokenized_chunks_masks_labels(
            tst_inputs, tst_masks, tst_labels)
        
        return tr_inputs, tr_masks, tr_labels, tr_imdb_ids, val_inputs, val_masks, val_labels, val_imdb_ids, tst_inputs, tst_masks, tst_labels, tst_imdb_ids


In [ ]:
def get_dataloader(input_ids, labels, attention_masks, batch_size=1,  
                   phase='train', sampler=None):
        dataset = TensorDataset(input_ids, attention_masks, labels)
        if phase=='train':
            sampler = sampler if not sampler is None else RandomSampler(dataset)
            dataloader = DataLoader(
                        dataset,  
                        batch_size = batch_size,
                        sampler = sampler,
                        drop_last = True
                    )
        else:
            dataloader = DataLoader(
                        dataset,  
                        batch_size = batch_size,
                        drop_last = True
                    )

        return dataloader 

In [ ]:
def get_data_loaders():
    SD = ScriptData(config, tokenizer)
    #tr_inputs, val_inputs, tst_inputs, tr_labels, val_labels, tst_labels = AD.get_train_val_split()
    #tr_inputs, tr_attention_masks, tr_labels, val_inputs, val_attention_masks, \
    #val_labels, tst_inputs, tst_attention_masks, tst_labels =SD.get_train_val_split()
    tr_inputs, tr_attention_masks, tr_labels, tr_imdb_ids, val_inputs, val_attention_masks, val_labels, val_imdb_ids, tst_inputs, tst_attention_masks, tst_labels, tst_imdb_ids =  SD.get_train_val_split()

    #tr_loader = get_dataloader(tr_inputs[[1,2,9,19]], tr_labels[[1,2,9,19]],  tr_attention_masks[[1,2,9,19]],
    tr_loader = get_dataloader(tr_inputs, tr_labels,  tr_attention_masks,
                            batch_size=config['train']['tr_batch_size'])
    val_loader = get_dataloader(val_inputs, val_labels,  val_attention_masks,
                                batch_size=config['train']['tst_batch_size'])
    tst_loader = get_dataloader(tst_inputs, tst_labels,  tst_attention_masks,
                                batch_size=config['train']['tst_batch_size'])
    
    return tr_loader, val_loader, tst_loader


### Check:

In [ ]:
SD = ScriptData(config, tokenizer)

In [ ]:
print(SD.imdb_ids)

In [ ]:
print(SD.labels[:10])

In [ ]:
tokenized_scripts, tokenized_chunks_for_scripts, attention_masks = SD.make_tokenized_chunks_for_scripts()

In [ ]:
#tr_inputs, tr_masks, tr_labels, val_inputs, val_masks, val_labels, tst_inputs, tst_masks, tst_labels = SD.get_train_val_split()
tr_inputs, tr_masks, tr_labels, tr_imdb_ids, val_inputs, val_masks, val_labels, val_imdb_ids, tst_inputs, tst_masks, tst_labels, tst_imdb_ids = SD.get_train_val_split()

In [ ]:
print(len(tr_imdb_ids))
print(len(tr_inputs))
print(len(val_imdb_ids))
print(len(val_inputs))
print(len(tst_imdb_ids))
print(len(tst_inputs))

In [ ]:
print(tr_imdb_ids)

In [ ]:
print(min(tr_inds))
print(max(tr_inds))

In [ ]:
print(min(val_inds))
print(max(val_inds))

In [ ]:
print(min(tst_inds))
print(max(tst_inds))

In [ ]:
imdb_ids_to_labels = {'tr': dict(zip(tr_imdb_ids, tr_labels)), 'val':dict(zip(val_imdb_ids, val_labels)), 'tst':dict(zip(tst_imdb_ids, tst_labels))}

In [ ]:
print(imdb_ids_to_labels)

In [ ]:
print(imdb_ids_to_labels['tr']['0086423'].item())

In [ ]:
examples_inds = {'train':tr_inds, 'val':val_inds, 'test':tst_inds}

In [ ]:
with open('meta_score_data_inds.pickle', 'wb') as f:
    pickle.dump(examples_inds, f)

In [ ]:
print(tr_labels[:20])
print(tr_labels[[1,2,9,19]])

In [ ]:
tr_loader, val_loader, tst_loader = get_data_loaders()

In [ ]:
for d in tr_loader:
    print(d)
    print(d[0].shape)
    print(d[1].shape)
    print(d[2].shape)
    break

### Scripts length stats:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt 

lengths = [len(x) for x in tokenized_scripts]
bins = list(range(0,250000,10000)) + [250000, 1000000]
h, b = np.histogram(lengths, bins=bins)
h_cum = np.cumsum(h)
nrof_movies = np.sum(h)
h_cum_to_show = [str(x)+ ':\n'+str(round(float(x)/nrof_movies, 3)) for x in h_cum]
file_path = os.path.join('/content/drive/MyDrive/NLP/Movie scripts dataset/BERT training data/train_scripts_length.png')
bins = [str(bins[i])+'-'+str(bins[i+1])for i in range(len(bins)-1)]
show_histogram(bins, h, figsize=(20,10), x_label='Number of tokens', values_to_show= h_cum_to_show, y_label='Number of scripts',
               set_rotation=True, title='Scripts length distribution', file_path=file_path,
               dpi=200, to_save=True)

## Model:

In [ ]:
class TransformerSA(nn.Module):
    def __init__(self, model, confg): 
        super(TransformerSA, self).__init__()
        self.transformer_model = model
        
        #for param in self.transformer_model.parameters():
         #   param.requires_grad = False

        #self.cls_ff = torch.nn.Linear(config['train']['embedding_size'], 1)
        '''
        self.cls_ff = nn.Sequential(
            nn.Linear(config['train']['embedding_size'], 3072),
            nn.Linear(3072, config['train']['embedding_size']),
            nn.LayerNorm((768,), eps=1e-12, elementwise_affine=True),
            nn.Dropout(0.1),
            nn.Linear(config['train']['embedding_size'], config['train']['embedding_size']),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(config['train']['embedding_size'], config['train']['num_classes']),
            nn.Sigmoid()
        )
        '''
        self.lstm = LSTM(config['train']['embedding_size'],config['train']['embedding_size'])
        self.cls_ff = nn.Linear(config['train']['embedding_size'], config['train']['num_classes'])
        #self.sigmoid = nn.Sigmoid()
        

    def forward(self, script, input_masks, label):
        '''
        script squeezed (of size (nrof chunks, max seq len))
        '''
        #script_vector = []
        bert_output = torch.zeros(config['train']['tr_batch_size'], 
                                  config['train']['max_scene_number'],
                                  #config['train']['max_script_length'] // config['train']['max_seq_length'],
                                  config['train']['embedding_size'],
                                  device=config['train']['device'])
        #print(bert_output.shape)
        #### ?????!
        for i in range(script.shape[1]):   
            bert_output[:, i, :] = self.transformer_model(script[:, i, :],
                                 attention_mask = input_masks[:, i, :])[1]
            '''
            outputs = self.transformer_model(script[:, i, :],
                                 attention_mask = input_masks[:, i, :], 
                                 labels=torch.LongTensor([0] * script.shape[0]).to(config['train']['device']),
                                 return_dict=True)  

            cls_vector = outputs['hidden_states'][-1][0][0]
            #print('CLS shape:', cls_vector.shape)
            bert_output[:, i, :] = cls_vector ### БЫЛО НЕПРАВИЛЬНО"!!!!!
            #script_vector.append(cls_vector)
            '''
        output, (_, _) = self.lstm(bert_output.permute(1,0,2))
        last_layer = output[-1]
        ff_res = self.cls_ff(last_layer)
        #stacked = torch.stack(script_vector)
        #script_vector = torch.mean(bert_output, 1)
        #ff_res = self.cls_ff(script_vector)

        return ff_res
        #return self.sigmoid(ff_res)



## Train:

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def plot_conf_matr(class_correct, class_total):
    results = np.zeros((2,2))
    results[0][0] += class_correct[0]
    results[1][1] += class_correct[1]
    results[0][1] += class_total[0] - class_correct[0]
    results[1][0] += class_total[1] - class_correct[1]

    df_cm = pd.DataFrame(results.astype(np.int), index = ['not nominated', 'nominated'],
                  columns = ['not nominated', 'nominated'])
    plt.figure(figsize = (7,7))    
    sn.heatmap(df_cm, annot=True, fmt='d')
    plt.savefig('conf_matrix.png', bbox_inches='tight')
    plt.close()
    
def plot_predictions(predictions, trues, file_name='', plot_point_count=200):        
    plt.figure(figsize = (7,7))    
    plt.scatter(trues[:plot_point_count], predictions[:plot_point_count], marker='.', s=20)
    plt.ylabel('Predicted')
    plt.xlabel('Actual')
    plt.title('Predicted vs true scores')
    plt.savefig(file_name + '.png', bbox_inches='tight')
    #plt.show()
    


In [ ]:
class Train():
    def __init__(self, config):
        self.config = config 
        transformer_model = BertModel.from_pretrained(
                config['train']['pretrained_model_type'],
                gradient_checkpointing=True) 
        '''
        transformer_model = self.config['train']['model'].from_pretrained(
                config['train']['pretrained_model_type'], 
                num_labels = self.config['train']['num_classes'],
                output_attentions = False, 
                output_hidden_states = True, 
                gradient_checkpointing=True
                )
        '''
        self.model = TransformerSA(transformer_model, self.config)
        
        opt_config = self.config['train']['optim']['AdamW']
        for key, val in opt_config.items():
            mlflow.log_param(key, val)
        mlflow.log_param('nrof_classes', self.config['train']['num_classes'])
        
        # попробовать SGD??
        self.optimizer = AdamW(self.model.parameters(),
                  lr = opt_config['lr']#, 
                  #eps = opt_config['eps'], 
                  #weight_decay=opt_config['weight_decay'],
                )        
        #'''
        #self.total_steps = self.config['train']['nrof_steps']
        #self.scheduler = get_cosine_schedule_with_warmup(self.optimizer, 
         #                                   num_warmup_steps = 10, 
          #                                  num_training_steps = self.total_steps)
        #self.crit = CrossEntropyLoss()
        #self.crit = nn.MSELoss()
        #self.crit = nn.BCELoss()
        self.crit = nn.BCEWithLogitsLoss()
        self.model.to(self.config['train']['device'])
        #self.training_stats = []

        self.global_step = 0

        #mlflow.log_param('total_steps', self.total_steps)
    
    def save_model(self):
        torch.save({"model": self.model.state_dict(),
                    "optimizer": self.optimizer.state_dict(),
                    #"scheduler": self.scheduler.state_dict(),
                    'global_step': self.global_step
                    },
                   os.path.join(self.config['paths']['ckpt_dir'], 
                                self.config['train']['exp_name'] + '_checkpoint'))
    
    def load_model(self):
        ckpt = torch.load(os.path.join(self.config['paths']['ckpt_dir'],
                                       self.config['train']['exp_name'] + '_checkpoint'),
                          map_location=self.config['train']['device'])
        self.global_step = ckpt["global_step"] + 1
        self.model.load_state_dict(ckpt["model"])   
        self.optimizer.load_state_dict(ckpt["optimizer"])
        #self.scheduler.load_state_dict(ckpt["scheduler"])
        print("Model loaded...")


    def train(self, train_dataloader, validation_dataloader):
        if self.config['train']['load_model']:
            self.load_model()
        self.model.train()
        t0 = time()

        predicts, trues = [], []
        tr_losses = []
        cur_loss, nrof_steps, = 0., 0,
        nrof_cor_predicts_current, nrof_cor_predicts =0, 0
        nrof_samples_current, nrof_samples = 0, 0
        print('GLOBAL STEP:', self.global_step)
        #class_correct = list(0. for i in range(self.config['train']['num_classes']))
        #class_total = list(0. for i in range(self.config['train']['num_classes']))
    
        #while self.global_step < self.total_steps:
        for epoch in range(self.config['train']['nrof_epochs']):
            predicts, trues, tr_losses = [], [], []
            for step, batch in enumerate(train_dataloader):
                if epoch * len(train_dataloader) + step < self.global_step:
                    continue
                #if epoch==0 and step<self.global_step:
                 #   continue
                print('step', step)
                print('epoch', epoch)
                b_input_ids = batch[0].to(self.config['train']['device'])
                b_input_mask = batch[1].to(self.config['train']['device'])
                b_labels = batch[2].to(self.config['train']['device'])

                
                self.model.zero_grad()  
                try:
                    output = self.model(b_input_ids, 
                                        b_input_mask, b_labels)
                    output = output.squeeze()
                    loss = self.crit(output, b_labels)
                    tr_losses.append(loss.item())
                    cur_loss +=tr_losses[-1]
                    #_, predicted = torch.max(output,-1)
                    predicted = output.sigmoid()  # should be size = 1?
                    predicts.extend(predicted.cpu().detach().numpy().flatten().tolist())
                    trues.extend(b_labels.cpu().detach().numpy().flatten().tolist())            
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                    self.optimizer.step()
                    #self.scheduler.step()
                    self.global_step+=1
                    nrof_steps+=1
                    nrof_samples+=len(b_labels)
                except RuntimeError as excp:
                    if "out of memory" in str(excp):
                        print("Out of memory!")
                        if hasattr(torch.cuda,'empty_cache'): 
                            torch.cuda.empty_cache() 
                    else:
                        raise excp

                
                if self.global_step % 5 == 0: 
                    elapsed = format_time(time() - t0)
                    print('Elapsed: {:}.'.format(elapsed))
                    try:
                        if self.global_step % 15 == 0: 
                            avg_val_loss, max_errors, val_predicts, val_trues  = self.validate(validation_dataloader)
                            mlflow.log_metric("val_loss", avg_val_loss, self.global_step)
                            mlflow.log_metric("val_max_error", max_errors, self.global_step)
                            plot_predictions(val_predicts, val_trues, file_name='val_predictions_'+str(self.global_step))
                            mlflow.log_artifact('val_predictions_'+str(self.global_step)+'.png')
                            self.model.train()
                    except RuntimeError as excp:
                        if "out of memory" in str(excp):
                            print("Out of memory!VAL")
                            if hasattr(torch.cuda,'empty_cache'): 
                                torch.cuda.empty_cache() 
                        else:
                            raise excp
                    
                    if nrof_steps!=0:
                        trues, predicts = trues[-50:], predicts[-50:] 
                        tr_losses = tr_losses[-50:]
                        max_errors = max_error(trues,predicts)
                        tr_av_loss = np.mean(tr_losses)
                        tr_loss = cur_loss / nrof_steps
                        #bal_accuracy = balanced_accuracy_score(trues, predicts)
                        #if self.global_step % 50 == 0: 
                         #   plot_predictions(predicts, trues, file_name='train_predictions_'+str(self.global_step))
                          #  mlflow.log_artifact('train_predictions_'+str(self.global_step)+'.png')
                        mlflow.log_metric("train_loss", tr_loss, self.global_step)
                        mlflow.log_metric("train_average_loss", tr_av_loss, self.global_step)
                        mlflow.log_metric("train_max_error", max_errors, self.global_step)
                        #mlflow.log_metric("train_bal_accuracy", bal_accuracy, self.global_step)
                        print('tr loss: {}\ntr average loss:{}\nmax errors:{}'.format(tr_loss, tr_av_loss, max_errors))
                        print('trues: {}\npredicts:{}'.format(trues, predicts))
                        cur_loss, nrof_steps = 0.,0
                        #trues, predicts = [], []
                        
                

                if self.global_step % 15 == 0: 
                    self.save_model()
                    try:
                        copytree('/content/mlruns', self.config['paths']['mlruns'] + '_' + self.config['train']['exp_name'] +'_'+str(self.global_step))
                    except Exception as e:
                        print(e)
                        pass 

            training_time = (time() - t0)
        
        
        self.save_model()
        try:
            copytree('/content/mlruns', self.config['paths']['mlruns'] + '_' + self.config['train']['exp_name'])
        except Exception as e:
            print(e)
            pass 
            


    def validate(self, validation_dataloader, to_load=False):
        if to_load:
            self.load_model()
        t1 = time()
        self.model.eval()
        predicts, trues = [], []
        nrof_steps, val_loss, nrof_cor_predicts, nrof_samples = 0, 0., 0, 0
        #class_correct = list(0. for i in range(self.config['train']['num_classes']))
        #class_total = list(0. for i in range(self.config['train']['num_classes']))

        for i, batch in tqdm(enumerate(validation_dataloader)):
            with torch.no_grad(): 
                b_input_ids = batch[0].to(self.config['train']['device'])
                b_input_mask = batch[1].to(self.config['train']['device'])
                b_labels = batch[2].to(self.config['train']['device'])
                output = self.model(b_input_ids, 
                                    b_input_mask, b_labels)
                output = output.squeeze()
                val_loss += self.crit(output, b_labels).item()
                #val_loss += self.crit(output, b_labels.unsqueeze(-1)).item()
                #_, predicted = torch.max(output,-1)
                predicted = output.sigmoid()
                predicts.extend(predicted.cpu().detach().numpy().flatten().tolist())
                trues.extend(b_labels.cpu().detach().numpy().flatten().tolist())            
                #if_right = int(predicted == b_labels)
                #nrof_cor_predicts += if_right
                #nrof_cor_predicts_current += if_right
                #class_correct[b_labels.item()] += if_right
                #class_total[b_labels.item()] += 1

                nrof_steps+=1
                nrof_samples+=len(b_labels)
                #nrof_samples_current+=len(b_labels)


                '''
                b_input_ids = batch[0].to(self.config['train']['device'])
                #b_input_mask = batch[1].to(self.config['train']['device'])
                b_labels = batch[1].to(self.config['train']['device'])
                _,_, = self.model(b_input_ids, 
                            token_type_ids=None, 
                            #attention_mask=b_input_mask,
                            labels=b_labels,
                            return_dict=True)
                val_loss += outputs.loss.item()
                _, predicted = torch.max(outputs.logits,-1)
                c = (predicted == b_labels)
                nrof_cor_predicts += c.sum().item()
                for i in range(b_labels.size(0)):
                    label = b_labels[i]
                    if torch.is_tensor(label):
                        for j, ll in enumerate(label):
                            class_correct[ll] += c[i][j].squeeze().item()
                            class_total[ll] += 1
                    else:
                        class_correct[label] += c[i].squeeze().item()
                        class_total[label] += 1

                nrof_samples += len(b_labels)
                '''

                
        #avg_val_accuracy = nrof_cor_predicts / nrof_samples
        #val_classes_accs = np.asarray(class_correct) / np.asarray(class_total)
        avg_val_loss = val_loss / nrof_steps
        max_errors = max_error(trues, predicts)
        #avg_bal_accuracy = balanced_accuracy_score(trues, predicts)
        validation_time = (time() - t1)
        
        print("  Validation took: {:}".format(validation_time))
        
        return avg_val_loss, max_errors, predicts, trues




In [ ]:
#tr_loader, val_loader, tst_loader = get_data_loaders()
T = Train(config)
T.train(tr_loader, val_loader)
#avg_val_accuracy, avg_val_loss, class_correct, class_total = T.validate(val_loader, to_load=True)
#print('val loss: {}\nval accuracy: {}'.format(avg_val_loss, avg_val_accuracy))
#plot_conf_matr(class_correct, class_total)

In [ ]:
#from shutil import copytree

copytree('/content/mlruns', '/content/drive/MyDrive/NLP/Movie scripts models/mlruns')

In [ ]:
!cp -r '/content/drive/MyDrive/NLP/Movie scripts models/BERTScorer/mlruns_script_scorer_360' .

In [ ]:
!cp -r '/content/mlruns' '/content/drive/MyDrive/NLP/Movie scripts models/BERTAssess/mlruns_script_scorer_batch_overfit'

In [ ]:
for d in val_loader:
    print(d[2].shape)
    break

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

with mlflow.start_run(run_name=config['train']['exp_name'], run_id='b5bfc0c937af4acebb9233469d310b66'):
    tr_loader, val_loader, tst_loader = get_data_loaders()
    T = Train(config)
    T.train(tr_loader, val_loader)

In [ ]:
mlflow.end_run()

In [ ]:
import shutil

shutil.rmtree('/content/mlruns')

In [ ]:
import mlflow
from pyngrok import ngrok

In [ ]:
!cp -r '/content/drive/MyDrive/NLP/Movie scripts models/BERTScorer/mlruns_script_scorer_360' .

In [ ]:
get_ipython().system_raw("mlflow ui --port 5000 &") # run tracking UI in the background
NGROK_AUTH_TOKEN = ""
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

In [ ]:
ngrok.kill()

# Regression (BoW + Linear regression):

In [ ]:
from sklearn import linear_model
from sklearn.metrics import explained_variance_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
print(len(os.listdir(config['paths']['script_lemmas'])))

## Data preparing:

In [ ]:
def get_train_test_split(tr_imdb_ids, val_imdb_ids, tst_imdb_ids, path_to_script_lemmas):
    processed_texts = []
    train_inds, val_inds, test_inds = [], [], []
    train_labels, train_imdb_ids, val_labels, val_imdb_ids, test_labels, test_imdb_ids = [], [], [], [], [], []

    i = 0
    for file_name in tqdm(os.listdir(path_to_script_lemmas)):
        imdb_id = file_name.split('_')[1]
        with open(os.path.join(path_to_script_lemmas, file_name), 'r') as f:
            text = f.read()
        processed_texts.append(text)
        if imdb_id in tr_imdb_ids:
            train_inds.append(i)
            train_labels.append(imdb_ids_to_labels['tr'][imdb_id].item())
            train_imdb_ids.append(imdb_id)
        elif imdb_id in val_imdb_ids:
            val_inds.append(i)
            val_labels.append(imdb_ids_to_labels['val'][imdb_id].item())
            val_imdb_ids.append(imdb_id)
        elif imdb_id in tst_imdb_ids:
            test_inds.append(i)
            test_labels.append(imdb_ids_to_labels['tst'][imdb_id].item())
            test_imdb_ids.append(imdb_id)
        i+=1
    return processed_texts, train_inds, val_inds, test_inds, train_labels, val_labels, test_labels, train_imdb_ids, val_imdb_ids, test_imdb_ids

In [ ]:
processed_texts, train_inds, val_inds, test_inds, train_labels, val_labels, test_labels, train_imdb_ids, val_imdb_ids, test_imdb_ids = get_train_test_split(tr_imdb_ids, val_imdb_ids, tst_imdb_ids, config['paths']['script_lemmas'])

In [ ]:
print(len(processed_texts))

In [ ]:
print(len(train_inds))

In [ ]:
print(train_labels[:20])

### Visualize data:

In [ ]:
h, b = np.histogram(train_labels + val_labels + test_labels)
#h, b = plt.hist(train_labels)
bins = [str(round(b[i], 2)) +'-' + str(round(b[i+1], 2)) for i in range(len(b)-1)]
show_histogram(bins, h, values_to_show=None, figsize=(20, 15), x_label='scores', y_label='number of movies',
                   set_rotation=True, title='Metacritic scores distriburion', file_path='meta_critic_scores_distribution.png', dpi=100, to_save=True, to_show=True)

In [ ]:
all_labels = train_labels + val_labels + test_labels

In [ ]:
scores_mean = np.mean(all_labels)
scores_median = np.median(all_labels)

In [ ]:
print(scores_mean)
print(scores_median)

In [ ]:
results_df = {'max_features':[], 'binary':[],'norm':[],'use_idf':[], 'smooth_idf':[],
              'sublinear_tf':[], 'train_r2_score':[], 'test_r2_score':[], 'test_evs':[],
              'test_r2_score_with_mean':[], 'test_r2_score_with_median':[], 
              'test_evs_with_mean':[], 'test_evs_with_median':[], 'test_mae':[],
                'test_mae_with_mean':[], 'test_mae_with_median':[]}

nrof_exps = 0
for max_features in (1500, 2000):
    for binary in (False, True):
        for norm in ('l1', 'l2'):
            for sublinear_tf in (False, True):
                for use_idf in (False, True):
                    if use_idf:
                        for smooth_idf in (False, True):
                            tfidfconverter = TfidfVectorizer(max_features=max_features, min_df=5, max_df=0.7, stop_words=stopwords.words('english'),
                                                            binary=binary,norm=norm,use_idf=use_idf,smooth_idf=smooth_idf,sublinear_tf=sublinear_tf)
                            X = tfidfconverter.fit_transform(processed_texts).toarray()
                            X_train = X[train_inds]
                            X_val = X[val_inds]
                            X_test = X[test_inds]
                            y_train = train_labels
                            y_val = val_labels
                            y_test = test_labels
                            #X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=11)
                            #X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=11)
                            reg = linear_model.SGDRegressor()
                            #clf = svm.LinearSVC(class_weight='balanced', random_state=11)
                            reg.fit(X_train, y_train)
                            test_r2_score_with_mean = r2_score(y_test, [scores_mean] * len(y_test))
                            test_r2_score_with_median = r2_score(y_test, [scores_median] * len(y_test))
                            #clf = MLPClassifier(random_state=11, max_iter=10, hidden_layer_sizes =[2500, 2000, 1000, 500, 100], verbose=True).fit(X_train, y_train)
                            y_pred = reg.predict(X_test)
                            tr_y_pred = reg.predict(X_train)
                            test_r2_score = reg.score(X_test, y_test)
                            train_r2_score = reg.score(X_train, y_train)
                            test_evs =  explained_variance_score(y_test, y_pred)
                            test_evs_with_mean = explained_variance_score(y_test, [scores_mean] * len(y_test))
                            test_evs_with_median = explained_variance_score(y_test, [scores_median] * len(y_test))
                            #bal_acc = balanced_accuracy_score(y_test, y_pred)
                            #tr_bal_acc = balanced_accuracy_score(y_train,tr_y_pred)
                            #f1score = f1_score(y_test,y_pred)
                            #explained_variance_score(y_true, y_pred)
                            #print(balanced_accuracy_score(y_test,y_pred))
                            #print(f1_score(y_test,y_pred))
                            results_df['max_features'].append(max_features)
                            results_df['binary'].append(binary)
                            results_df['norm'].append(norm)
                            results_df['use_idf'].append(use_idf)
                            results_df['smooth_idf'].append(use_idf)
                            results_df['sublinear_tf'].append(sublinear_tf)
                            results_df['test_r2_score_with_mean'].append(test_r2_score_with_mean)
                            results_df['test_r2_score_with_median'].append(test_r2_score_with_median)
                            results_df['test_r2_score'].append(test_r2_score)
                            results_df['train_r2_score'].append(train_r2_score)
                            results_df['test_evs'].append(test_evs)
                            results_df['test_evs_with_mean'].append(test_evs_with_mean)
                            results_df['test_evs_with_median'].append(test_evs_with_median)
                            results_df['test_mae'].append(median_absolute_error(y_test, y_pred))
                            results_df['test_mae_with_mean'].append(median_absolute_error(y_test, [scores_mean] * len(y_test)))
                            results_df['test_mae_with_median'].append(median_absolute_error(y_test, [scores_median] * len(y_test)))
                            #results_df['Train balanced accuracy'].append(round(tr_bal_acc, 4))
                            #results_df['Test balanced accuracy'].append(round(bal_acc, 4))
                            #results_df['Test f1-score'].append(round(f1score, 4))
                            #plot_conf_matr(y_test,y_pred, classes_names=['minor', 'main'], title=)
                            #results_df['SVM linear results'].append('Train balanced accuracy: {:.4f}\nTest balanced accuracy: {:.4f}\nTest f1-score: {:.4f}'.format(tr_bal_acc, bal_acc, f1score))
                            nrof_exps+=1
                            print('nrof exps', nrof_exps)
                    else:
                        tfidfconverter = TfidfVectorizer(max_features=max_features, min_df=5, max_df=0.7, stop_words=stopwords.words('english'),
                                                            binary=binary,norm=norm,use_idf=use_idf,sublinear_tf=sublinear_tf)
                        X = tfidfconverter.fit_transform(processed_texts).toarray()
                        X_train = X[train_inds]
                        X_val = X[val_inds]
                        X_test = X[test_inds]
                        y_train = train_labels
                        y_val = val_labels
                        y_test = test_labels
                        #X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=11)
                        #X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=11)
                        reg = linear_model.SGDRegressor()
                        #clf = svm.LinearSVC(class_weight='balanced', random_state=11)
                        reg.fit(X_train, y_train)
                        #clf = MLPClassifier(random_state=11, max_iter=10, hidden_layer_sizes =[2500, 2000, 1000, 500, 100], verbose=True).fit(X_train, y_train)
                        y_pred = reg.predict(X_test)
                        tr_y_pred = reg.predict(X_train)
                        test_r2_score_with_mean = r2_score(y_test, [scores_mean] * len(y_test))
                        test_r2_score_with_median = r2_score(y_test, [scores_median] * len(y_test))
                        train_r2_score = reg.score(X_train, y_train)
                        test_evs =  explained_variance_score(y_test, y_pred)
                        test_evs_with_mean = explained_variance_score(y_test, [scores_mean] * len(y_test))
                        test_evs_with_median = explained_variance_score(y_test, [scores_median] * len(y_test))
                        #bal_acc = balanced_accuracy_score(y_test,y_pred)
                        #tr_bal_acc = balanced_accuracy_score(y_train,tr_y_pred)
                        #f1score = f1_score(y_test,y_pred)
                        #print(balanced_accuracy_score(y_test,y_pred))
                        #print(f1_score(y_test,y_pred))
                        results_df['max_features'].append(max_features)
                        results_df['binary'].append(binary)
                        results_df['norm'].append(norm)
                        results_df['use_idf'].append(use_idf)
                        results_df['smooth_idf'].append(False)
                        results_df['sublinear_tf'].append(sublinear_tf)
                        results_df['test_r2_score_with_mean'].append(test_r2_score_with_mean)
                        results_df['test_r2_score_with_median'].append(test_r2_score_with_median)
                        results_df['test_r2_score'].append(test_r2_score)
                        results_df['train_r2_score'].append(train_r2_score)
                        results_df['test_evs'].append(test_evs)
                        results_df['test_evs_with_mean'].append(test_evs_with_mean)
                        results_df['test_evs_with_median'].append(test_evs_with_median)
                        results_df['test_mae'].append(median_absolute_error(y_test, y_pred))
                        results_df['test_mae_with_mean'].append(median_absolute_error(y_test, [scores_mean] * len(y_test)))
                        results_df['test_mae_with_median'].append(median_absolute_error(y_test, [scores_median] * len(y_test)))
                        #results_df['Train balanced accuracy'].append(round(tr_bal_acc, 4))
                        #results_df['Test balanced accuracy'].append(round(bal_acc, 4))
                        #results_df['Test f1-score'].append(round(f1score, 4))
                        #results_df['SVM linear results'].append('Train balanced accuracy: {:.4f}\nTest balanced accuracy: {:.4f}\nTest f1-score: {:.4f}'.format(tr_bal_acc, bal_acc, f1score))
                        nrof_exps+=1
                        print('nrof exps', nrof_exps)

In [ ]:
results_df = pd.DataFrame(results_df)

In [ ]:
results_df.to_excel('BoW_validation_results_metascores.xlsx', index=False)

In [ ]:
results_df

In [ ]:
tfidfconverter = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'),
                                                            binary=False,norm='l2',use_idf=True,smooth_idf = True, sublinear_tf=False)
X = tfidfconverter.fit_transform(processed_texts).toarray()
X_train = X[train_inds]
X_val = X[val_inds]
X_test = X[test_inds]
y_train = train_labels
y_val = val_labels
y_test = test_labels
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=11)
#X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=11)
reg = linear_model.SGDRegressor()
#clf = svm.LinearSVC(class_weight='balanced', random_state=11)
reg.fit(X_train, y_train)
#test_r2_score_with_mean = r2_score(y_test, [scores_mean] * len(y_test))
#test_r2_score_with_median = r2_score(y_test, [scores_median] * len(y_test))
#clf = MLPClassifier(random_state=11, max_iter=10, hidden_layer_sizes =[2500, 2000, 1000, 500, 100], verbose=True).fit(X_train, y_train)
y_pred = reg.predict(X_test)
tr_y_pred = reg.predict(X_train)

In [ ]:
print(X_test)

In [ ]:
nrof_texts_to_write = 50

In [ ]:
for i, text in enumerate(processed_texts):
    if i in train_inds:
        label = train_labels[train_inds.index(i)]
        imdb_id = train_imdb_ids[train_inds.index(i)]
        with open(os.path.join(config['paths']['meta_score_exampels'], str(round(label * 100))+'_'+str(imdb_id)+'.txt'), 'w') as f:
                  f.write(text)

